## Step 1 : Install the Dependencies


In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 51kB/s 
     |████████████████████████████████| 3.0MB 52.0MB/s 
     |████████████████████████████████| 419kB 52.3MB/s 


In [2]:
!pip install pandas-datareader

## Step 2: Importing project dependencies

In [3]:
import math                                 ## implement sigmoid function
import random                               ## Generate random number
import numpy as np                          ## Work with matrices and numpy array
import pandas as pd                         ## Data manipulation and analysis
import tensorflow as tf                     ## Creating neural network
import matplotlib.pyplot as plt             ## Data visualization
import pandas_datareader as data_reader     ## Work with stock market information

from tqdm import tqdm_notebook, tqdm        ## Visulaize any process in the project 
from collections import deque               ## Data structure

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
tf.__version__    ## Check TensorFlow version

'2.0.0-alpha0'

## Step 3: Building the AI Trader network

In [0]:
class AI_Trader():    ## Create AITrader class
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"):    ## action_space = {Stay, Buy and Sell}
    
    self.state_size = state_size        ## Number of input
    self.action_space = action_space    ## Number of output
    self.memory = deque(maxlen=2000)    ## Number of element to be stored inside
    self.inventory = []                 ## Store all the stock that we bought
    self.model_name = model_name
    
    self.gamma = 0.95                   ## Maximize the current reward over long time reward
    self.epsilon = 1.0                  ## Determine whether should we choose random action or use model for it (1 -> random)
    self.epsilon_final = 0.01           ## This is to make sure our model still do environment exploration
    self.epsilon_decay = 0.995          ## Rate of epsilon decay
    
    self.model = self.model_builder()   ## Execute model builder and store the model into self.model
    
  def model_builder(self):  ## Create a fully connected layers
    
    model = tf.keras.models.Sequential()    ## Define the model
    
    ## The architecture is 3 hidden layers with 32, 64 and 128 neuron corresponding
    ## All of them using ReLU activation function

    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))  
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    ## Output Layer
    ## Use Linear activation function because we are going to use mean square error for our loss
    ## We will modify our action with our reward which is a continuous number not a class

    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))    
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001)) ## mse -> mean Squared Error
    
    return model
  
  def trade(self, state):   ## Take a state as an input and output the action that should be taken
                            ## Should we use random generated action or use our model to perform an action
    
    if random.random() <= self.epsilon:   ## Use random number is random generated number leass than epsilon
      return random.randrange(self.action_space)  
    
    actions = self.model.predict(state)   ## Else use our model
    return np.argmax(actions[0])          ## Only return action with highest probablility
  
  
  def batch_train(self, batch_size):    ## Custom training function 
                                        ## Takes number of batch sizes as input
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):    ## Sample from the end of memory
                                                                            ## End index is length of memory
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:  ## Make sure it is not in terminal state
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0]) ## amax return maximum value of input array
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0) ## verbose -> do not want to print all training result
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay  ## decay the epsilon

## Step 4: Dataset preprocessing

### Defining helper functions

#### Sigmoid

In [0]:
def sigmoid(x):   ## Create custom sigmoid function
  return 1 / (1 + math.exp(-x))

#### Price format function

In [0]:
def stocks_price_format(n):   ## return the price format properly
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Dataset loader

In [0]:
def dataset_loader(stock_name):   ## Function that take stock name as input and output the closing price 
  
  ## Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")   ## Read the data from Yahoo!
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']  ## Get the closing price only

  return close

### State creator

In [0]:
def state_creator(data, timestep, window_size): ## Create function that take the data and output the state
  
  starting_id = timestep - window_size + 1    ## We need to start from 1 bcause of the way we structure the data
  
  if starting_id >= 0: 
    windowed_data = data[starting_id:timestep+1]  ## Extract the windowed data
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):  ## -1 because we have differences between current element and one after
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))  ## Normalize the next day and current day
    
  return np.array([state])    ## Return the numpy array

### Loading a dataset

In [0]:
stock_name = "AAPL"   ## Use Apple stock for this experiment
data = dataset_loader(stock_name)

## Stage 5: Training the AI Trader

### Setting hyper parameters

In [0]:
window_size = 10  ## Use 10 previous days to predict current one
episodes = 1000   ## Same as epochs

batch_size = 32 
data_samples = len(data) - 1  ## -1 because we cannot use the last one

### Defining the Trader model

In [0]:
trader = AI_Trader(window_size)   ## Pass the window size

In [13]:
trader.model.summary()  ## Check the neural network

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training loop

In [14]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes)) ## To show at which episode we are currently at
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):   ## TQDM is for visulaization
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)  ## Generating the next state 
    reward = 0
    
    if action == 1:   ## If we buy the stock
      trader.inventory.append(data[t])    ## Append the data into our inventory
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: ## If we sell the stock, need to check whether if we have any stock
      buy_price = trader.inventory.pop(0) ## Taking the first stock price instead of the last 
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price   ## Calculate the profit 
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:   ## Check whether it is the end of out samples
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))   ## Pass all the data into memory
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:   
      trader.batch_train(batch_size)    ## Call the batch_train
      
  if episode % 10 == 0:   ## Every 10 episodes 
    trader.model.save("ai_trader_{}.h5".format(episode))    ## Save the model, h5 is extension of the weight
    

  0%|          | 0/2482 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 30.572857
AI Trader sold:  $ 30.625713  Profit: $ 0.052856
AI Trader bought:  $ 30.282858
AI Trader bought:  $ 29.674286
AI Trader bought:  $ 30.092857
AI Trader bought:  $ 30.719999
AI Trader sold:  $ 28.250000  Profit: - $ 2.032858
AI Trader sold:  $ 29.010000  Profit: - $ 0.664286
AI Trader bought:  $ 29.420000
AI Trader bought:  $ 28.469999
AI Trader bought:  $ 27.437143
AI Trader bought:  $ 27.818571
AI Trader bought:  $ 27.980000
AI Trader sold:  $ 28.461428  Profit: - $ 1.631430
AI Trader bought:  $ 27.731428
AI Trader sold:  $ 28.381428  Profit: - $ 2.338572
AI Trader sold:  $ 29.057142  Profit: - $ 0.362858
AI Trader sold:  $ 28.935715  Profit: $ 0.465715
AI Trader bought:  $ 28.990000


  1%|▏         | 34/2482 [00:01<04:27,  9.15it/s]

AI Trader bought:  $ 28.151428


  1%|▏         | 35/2482 [00:02<07:42,  5.29it/s]

...

AI Trader sold:  $ 59.599998  Profit: $ 1.677143


 34%|███▍      | 840/2482 [03:36<07:09,  3.82it/s]

AI Trader sold:  $ 65.522858  Profit: $ 7.182858


 34%|███▍      | 846/2482 [03:37<06:58,  3.91it/s]

KeyboardInterrupt: ignored